# SIT744 Practical 6: Introducing ConvNet

*Dr Wei Luo*

<div class="alert alert-info">
We suggest that you run this notebook using Google Colab.
</div>

## Learning objectives

- Construct and train a Convolutional Neural Network
- Regularise training with data augmentation and dropout

## Pre-practical readings

- [Introduction to convnets](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.1-introduction-to-convnets.ipynb)
- [Using convnets with small datasets](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.2-using-convnets-with-small-datasets.ipynb)

## Task 1 Training a convnet

We will use the cats vs. dogs dataset in the textbook to show how to build a ConvNet to classify cat vs dog photos.

### Task 1.1 Prepare dataset

 As the dataset is available in `tfds`, we will get it from there.

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

TRAIN_DS_SIZE = 2000
VALID_DS_SIZE = 1000
TEST_DS_SIZE = 1000

(cat_dog_train, cat_dog_valid, cat_dog_test), info = tfds.load('cats_vs_dogs', 
                                                               split=[f'train[:{TRAIN_DS_SIZE}]', 
                                                                      f'train[{TRAIN_DS_SIZE}:{TRAIN_DS_SIZE + VALID_DS_SIZE}]', 
                                                                      f'train[{TRAIN_DS_SIZE + VALID_DS_SIZE}:{TRAIN_DS_SIZE + VALID_DS_SIZE + TEST_DS_SIZE}]'],
                                                               with_info=True,
                                                               as_supervised=True)


In [0]:
info

You can display some images using the function from the last practical.

In [0]:
## Show the image and label
import matplotlib.pyplot as plt
import numpy as np

def show(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(label)
  plt.axis('off')

for image, label in cat_dog_valid.take(5):
  show(image, label.numpy())

**exercise** 

1. Are the images of equal size? What is the value range of each pixel?
2. What happens if you change the argument `as_supervised` to `False`?
3. Check if the two classes are balanced in the data splits.

Last week, we saw how to use `map` function to preprocess images. 

In [0]:
IMAGE_SIZE = 150
def pre_process_image(image, label):
  image = tf.image.convert_image_dtype(image, tf.float32) ## Instead of manually scale the image, call a `tf.image` tool
  image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  return image, label

TRAIN_BATCH_SIZE = 20
train_batches = cat_dog_train.map(pre_process_image).batch(TRAIN_BATCH_SIZE).cache().repeat()
validation_batches = cat_dog_valid.map(pre_process_image).batch(TRAIN_BATCH_SIZE).cache().repeat()

**question** Why do we add `repeat()` at the end of the pipelines?


### Task 1.2 Build a convnet

We follow the common practice of using multiple blocks of Conv2D+MaxPooling2D layers. The number of channels was increased from 32 to 128, to form the visual features. 
In the end, two dense layers were used to perform the classification based on the convolutional features.

In [0]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers



def make_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu',
                          input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

  return model

model = make_model()  
model.summary()

Now we can feed the input pipelines to the model's `fit` method.
As we used `repeat()` in the pipelines, we need to define what an **epoch** means. This is done through the `steps_per_epoch` argument.

In [0]:
%%time

model = make_model()  

history = model.fit(
      train_batches,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_batches,
      validation_steps=50
      )

model.save('cats_and_dogs_small_1.h5')

We will use a nicer-looking plot function to view the training curves.

In [0]:
%pip install git+https://github.com/tensorflow/docs

In [0]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots

plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"": history}, metric = "acc")
plt.title("Accuracy")
plt.ylim([0,1])

Overfitting is more obvious from the plot of loss functions. 

In [0]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"": history}, metric = "loss")
plt.title("Accuracy")
plt.ylim([0,1])

Let's look at how the model performs on the test set.

In [0]:
model.evaluate(cat_dog_test.map(pre_process_image).batch(TRAIN_BATCH_SIZE))

In [0]:
for images, labels in cat_dog_test.map(pre_process_image).batch(TRAIN_BATCH_SIZE).take(1):
  predictions = model.predict(images) 
  for image, prediction in zip(images, predictions):
    show(image, np.round(prediction))

**exercise** We used 3x3 convolution filters in the above example. Try increase the size of the convolution filters (particular at the first convolution layer) and see how the training and testing accuracy change. You may change the `strides` argument for larger filters. 

## Task 2 Regularise the training


### Task 2.1 Using data augmentation

In Week 5, we learned that two ways to address overfitting include data augmentation and regularisation. In training convnet, data augmentation often means generating randomly transformed images from the existing training set. We can use image transformations provided in `tf.image`. 

In [0]:
for images, labels in cat_dog_train.take(1):
  show(image, label.numpy())

  show(tf.image.flip_left_right(image), label="Flip the image left to right") 

  show(tf.image.rot90(image), "Rotate an image by 90 degrees")

  show(tf.squeeze(tf.image.rgb_to_grayscale(image)), "Grayscale the image")

  show(tf.image.adjust_saturation(image, 3), "Saturate an image")

  show(tf.image.adjust_brightness(image, 0.4), "Change the brightness")

  show(tf.image.central_crop(image, central_fraction=0.5), "Center crop the image")


**exercise** Follow the example above to 

- flip the image upside down
- change the image with random brightness
- change the image with random contrast
- randomly crop the image


Now let's add random transformations to the input pipeline.

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def augment(image,label):
  image = tf.image.convert_image_dtype(image, tf.float32) 
  image = tf.image.random_flip_left_right(image)
  image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  image = tf.image.resize_with_pad(image, IMAGE_SIZE + 6, IMAGE_SIZE + 6) # Add pixels of padding
  image = tf.image.random_crop(image, size=[IMAGE_SIZE, IMAGE_SIZE, 3]) # Random crop back to IMAGE_SIZExIMAGE_SIZE
  return image, label

augmented_train_batches = (
    cat_dog_train
    .cache()
    # The augmentation is added here.
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(TRAIN_BATCH_SIZE)
    .repeat()
    .prefetch(AUTOTUNE)
)


Retrain the model with the augmented data.

In [0]:
%%time

model = make_model()

aug_history = model.fit(
      augmented_train_batches,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_batches,
      validation_steps=100
      )

model.save('cats_and_dogs_small_augmented.h5')

In [0]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({ "Augmented": aug_history, "Non-Augmented": history}, metric = "acc")
plt.title("Accuracy")
plt.ylim([0,1])

In [0]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({ "Augmented": aug_history, "Non-Augmented": history}, metric = "loss")
plt.title("Loss")
plt.ylim([0,1])

As we can see, the augmented data reduced overfitting.


**exercise** Change the example above to incorporate the following types of augmentations:

- Randomly flip an image vertically (upside down)
- Randomly rotate an image
- Randomly adjust the brightness, hue, saturation, or contrast

### Task 2.2 Add dropout

In week 4, we introduced dropout as a regularisation measure. It is often applied between dense layers.

In [0]:
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu',
                          input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

Retrain with both data augmentation and dropout.

In [0]:
dropout_history = model.fit(
      augmented_train_batches,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_batches,
      validation_steps=100
      )

model.save('cats_and_dogs_small_augmented.h5')

In [0]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"Augmented": aug_history, "Non-Augmented": history, "Dropout": dropout_history}, metric = "acc")
plt.title("Accuracy")
plt.ylim([0,1])

In [0]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({ "Augmented": aug_history, "Non-Augmented": history, "Dropout": dropout_history}, metric = "loss")
plt.title("Loss")
plt.ylim([0,1])

As you can see that overfitting has been further reduced.

**exercise** Try different dropout rates and see how they impact on the training and validation accuracy.

## Additional resources

- TensorFlow tutorial on [image classification](https://www.tensorflow.org/tutorials/images/classification)
- TensorFlow tutorial on [data augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation)